In [2]:
#GDD calculation for Soy from class SoyGDD. 
import pandas as pd

def fahrenheit_to_celsius(temp_f):
    return (temp_f - 32) * 5 / 9

def celsius_to_fahrenheit(temp_c):
    return (temp_c * 9 / 5) + 32

class SoyGDD:
    def __init__(self, daily_temp_df, time_column, min_temp_column, max_temp_column, celsius=True):
        self.daily_temp_df = daily_temp_df
        self.time_column = time_column
        self.min_temp_column = min_temp_column
        self.max_temp_column = max_temp_column
        self.celsius = celsius

        # Ensure the time column is of datetime type
        self.daily_temp_df[self.time_column] = pd.to_datetime(self.daily_temp_df[self.time_column])

        # If initial data is in Celsius, optionally convert it to Fahrenheit for uniform GDD calculation
        if self.celsius:
            self.daily_temp_df[self.min_temp_column] = self.daily_temp_df[self.min_temp_column].apply(celsius_to_fahrenheit)
            self.daily_temp_df[self.max_temp_column] = self.daily_temp_df[self.max_temp_column].apply(celsius_to_fahrenheit)
            self.celsius = False  # Now all data is in Fahrenheit

    def soy_gdd(self, start_date, end_date):
        start_date = pd.Timestamp(start_date)
        end_date = pd.Timestamp(end_date)

        date_mask = (self.daily_temp_df[self.time_column] >= start_date) & (self.daily_temp_df[self.time_column] <= end_date)
        masked_data = self.daily_temp_df.loc[date_mask].copy()

        gdd_values = []
        cumulative_gdd = 0

        for index, row in masked_data.iterrows():
            min_temp = max(row[self.min_temp_column], 50)  # Fahrenheit base
            max_temp = min(row[self.max_temp_column], 86)  # Fahrenheit upper limit

            mean_temp = (min_temp + max_temp) / 2
            base_temp = 50  # Fahrenheit base temperature for GDD calculation
            gdd = max(mean_temp - base_temp, 0)

            gdd_values.append(gdd)
            cumulative_gdd += gdd

        masked_data['GDD'] = gdd_values
        masked_data['Cumulative_GDD'] = masked_data['GDD'].cumsum()

        return masked_data

# This modified class converts temperatures to Fahrenheit if they start in Celsius, then computes and returns GDD in Fahrenheit.


Import min/max temperature from GEMS exchange

In [4]:
import pandas as pd
StPaul_GEMS = pd.read_csv("st_paul_gems_2023_2024.csv", on_bad_lines='skip')

In [5]:
StPaul_GEMS_2023 = SoyGDD(StPaul_GEMS, time_column='datetime', min_temp_column='min_temp', max_temp_column='max_temp')

In [6]:
#Using year 2024 start date and end date for mine gdd
start_date = pd.to_datetime("5-18-2023").date()
end_date = pd.to_datetime("10-20-2023").date()

Calculing 2023 Soy AGDD for StPaul

In [9]:
Soy_gdd_StPaulGEMS2023 = StPaul_GEMS_2023.soy_gdd(start_date, end_date)
print(Soy_gdd_StPaulGEMS2023)

     clouds   datetime  dewpt  dhi  dni  ghi  max_dhi  max_dni  max_ghi  \
137      70 2023-05-18    9.8   56  451  340      121      920      945   
138      54 2023-05-19    3.3   56  452  341      121      920      946   
139       9 2023-05-20    4.7   56  453  342      121      920      948   
140      16 2023-05-21    6.3   56  454  343      121      920      949   
141      27 2023-05-22    7.5   57  455  345      121      920      951   
..      ...        ...    ...  ...  ...  ...      ...      ...      ...   
288      19 2023-10-16    4.8   35  296  158      102      834      584   
289      38 2023-10-17    4.3   35  293  156      101      832      578   
290      70 2023-10-18    6.5   34  291  153      101      830      573   
291      85 2023-10-19    8.5   34  288  151      101      828      567   
292      25 2023-10-20    7.2   34  286  149      100      826      561   

     max_temp  ...  t_solar_rad  temp          ts  wind_dir  wind_gust_spd  \
137     66.02  ...   